In [6]:
import numpy as np
import scipy.constants as _c

from onix.models.schiff import MeasurementSettings, Atom, NewPhysicsSensitivities
from onix.units import ureg, Q_

## Effective electric field

In [55]:
def polarizability_atomic_unit_to_SI(polarizability_au):
    """Converts a polarizability from atomic units to SI units.

    See https://link.aps.org/doi/10.1103/PhysRevA.83.012503, text below Eq. 13.
    """
    a0 = _c.physical_constants["Bohr radius"][0]
    polarizability_au_to_SI = 4 * np.pi * _c.epsilon_0 * a0**3
    return polarizability_au * polarizability_au_to_SI * ureg.A**2 * ureg.s**4 / ureg.kg 

def Stark_shift_coefficient(polarizability_SI, electric_field):
    return polarizability_SI * electric_field * 2

def Stark_shift(polarizability_SI, electric_field):
    return polarizability_SI * electric_field ** 2

### from the Stark shift coefficient and a conservative 100 a.u. dynamic scalar staic polarizability.

In [88]:
measured_Stark_coefficient = 27.18 * ureg.kHz / (ureg.V / ureg.cm)
dssp_au = 100  # very conservative
dssp_SI = polarizability_atomic_unit_to_SI(dssp_au)
electric_field = 0.547 * ureg.MV / ureg.cm
Stark_coefficient_from_dssp = Stark_shift_coefficient(dssp_SI, electric_field) / ureg.planck_constant
print(f"The calculated Stark shift coefficient is {Stark_coefficient_from_dssp.to('kHz/(V/cm)'):.1f} at {electric_field} and {dssp_au} a.u. DSSP.")
print(f"The measured Stark shift coefficient is {measured_Stark_coefficient.to('kHz/(V/cm)'):.1f}.")

The calculated Stark shift coefficient is 27.2 centimeter * kilohertz / volt at 0.547 megavolt / centimeter and 100 a.u. DSSP.
The measured Stark shift coefficient is 27.2 centimeter * kilohertz / volt.


### from the Stark shift coefficient and a reasonable -1 a.u. dynamic scalar staic polarizability.

In [107]:
measured_Stark_coefficient = 27.18 * ureg.kHz / (ureg.V / ureg.cm)
dssp_au = -1  # reasonable, flip sign to make the Stark shift sign correct.
dssp_SI = polarizability_atomic_unit_to_SI(dssp_au)
electric_field = 54.7 * ureg.MV / ureg.cm
Stark_coefficient_from_dssp = Stark_shift_coefficient(dssp_SI, electric_field) / ureg.planck_constant
print(f"The calculated Stark shift coefficient is {Stark_coefficient_from_dssp.to('kHz/(V/cm)'):.1f} at {electric_field} and {dssp_au} a.u. DSSP.")
print(f"The measured Stark shift coefficient is {measured_Stark_coefficient.to('kHz/(V/cm)'):.1f}.")

The calculated Stark shift coefficient is -27.2 centimeter * kilohertz / volt at 54.7 megavolt / centimeter and -1 a.u. DSSP.
The measured Stark shift coefficient is 27.2 centimeter * kilohertz / volt.


### from the optical Stark shift and a reasonable -1 a.u. dynamic scalar staic polarizability.

In [108]:
# from the optical Stark shift from free-space Eu transition frequency
Eu_optical_free_space = 17270 * ureg.cm**-1 * ureg.c * ureg.planck_constant
Eu_optical_YSO = 516847.5 * ureg.GHz * ureg.planck_constant
Eu_Stark_shift = Eu_optical_YSO - Eu_optical_free_space
print(f"The calculated Stark shift is {(Stark_shift(dssp_SI, electric_field) / ureg.planck_constant).to('GHz'):.0f} at {electric_field} and {dssp_au} a.u. DSSP.")
print(f"The actual Stark shift is {Eu_Stark_shift / ureg.planck_constant}.")

The calculated Stark shift is -745 gigahertz at 54.7 megavolt / centimeter and -1 a.u. DSSP.
The actual Stark shift is -894.0749660000438 gigahertz.


###  from the hyperfine Stark shift and shift coefficient between a and b states

In [101]:
# select DSSP and electric field so that both the hyperfine splitting and Stark shift coefficient agrees with experiment.
# This is only approximate as the electric field affects both the D and E terms in the Q tensor,
# so the splitting is not perfectly quadratic with the electric field.
Eu_Stark_ab = 119 * ureg.MHz * ureg.planck_constant
Eu_Stark_coefficient_ab = 1.0 * ureg.Hz * ureg.cm / ureg.V * 2.58  # PhysRevLett.113.157603, scaled to 153Eu
dssp_ab_au = 0.00005
dssp_ab_SI = polarizability_atomic_unit_to_SI(dssp_ab_au)
electric_field_ab = 100 * ureg.MV / ureg.cm
Stark_coefficient_from_dssp_ab = Stark_shift_coefficient(dssp_ab_SI, electric_field_ab) / ureg.planck_constant
print(f"The calculated Stark shift coefficient is {Stark_coefficient_from_dssp_ab.to('Hz/(V/cm)'):.1f} at {electric_field_ab} and {dssp_ab_au} a.u. DSSP.")
print(f"The measured Stark shift coefficient is {Eu_Stark_coefficient_ab.to('Hz/(V/cm)'):.1f}.")
print(f"The calculated Stark shift is {(Stark_shift(dssp_ab_SI, electric_field_ab) / ureg.planck_constant).to('MHz'):.0f} at {electric_field_ab} and {dssp_ab_au} a.u. DSSP.")
print(f"The actual Stark shift is {Eu_Stark_ab / ureg.planck_constant}.")

The calculated Stark shift coefficient is 2.5 centimeter * hertz / volt at 100.0 megavolt / centimeter and 5e-05 a.u. DSSP.
The measured Stark shift coefficient is 2.6 centimeter * hertz / volt.
The calculated Stark shift is 124 megahertz at 100.0 megavolt / centimeter and 5e-05 a.u. DSSP.
The actual Stark shift is 119.0 megahertz.


###  from the hyperfine Stark shift and shift coefficient between b and c states

In [110]:
# select DSSP and electric field so that both the hyperfine splitting and Stark shift coefficient agrees with experiment.
# This is only approximate as the electric field affects both the D and E terms in the Q tensor,
# so the splitting is not perfectly quadratic with the electric field.
Eu_Stark_bc = 89 * ureg.MHz * ureg.planck_constant
Eu_Stark_coefficient_bc = 0.43 * ureg.Hz * ureg.cm / ureg.V * 2.58  # PhysRevLett.113.157603, scaled to 153Eu
dssp_bc_au = 0.000015
dssp_bc_SI = polarizability_atomic_unit_to_SI(dssp_bc_au)
electric_field_bc = 150 * ureg.MV / ureg.cm
Stark_coefficient_from_dssp_bc = Stark_shift_coefficient(dssp_bc_SI, electric_field_bc) / ureg.planck_constant
print(f"The calculated Stark shift coefficient is {Stark_coefficient_from_dssp_bc.to('Hz/(V/cm)'):.1f} at {electric_field_bc} and {dssp_bc_au} a.u. DSSP.")
print(f"The measured Stark shift coefficient is {Eu_Stark_coefficient_bc.to('Hz/(V/cm)'):.1f}.")
print(f"The calculated Stark shift is {(Stark_shift(dssp_bc_SI, electric_field_bc) / ureg.planck_constant).to('MHz'):.0f} at {electric_field_bc} and {dssp_bc_au} a.u. DSSP.")
print(f"The actual Stark shift is {Eu_Stark_bc / ureg.planck_constant}.")

The calculated Stark shift coefficient is 1.1 centimeter * hertz / volt at 150.0 megavolt / centimeter and 1.5e-05 a.u. DSSP.
The measured Stark shift coefficient is 1.1 centimeter * hertz / volt.
The calculated Stark shift is 84 megahertz at 150.0 megavolt / centimeter and 1.5e-05 a.u. DSSP.
The actual Stark shift is 89.0 megahertz.


Dependent on the method and approximation used, the result effective E field range from 0.5 to 150 MV / cm. The most reasonable approximation is probably assuming the optical DSSP as -1 a.u., where the Stark shift coefficient and Stark shift both approximately agree with the experiment at 50 MV / cm E field. This 50 MV / cm number is used in the calculations below.

### Electric field along the nuclear spin

In [109]:
effective_E_field = 50 * ureg.MV / ureg.cm
hat_I_dot_hat_n_b = 0.3  # b state
effective_E_field_along_I_b = effective_E_field * hat_I_dot_hat_n_b

## Europium atom

In [112]:
K_S = 3.8
K_Z = 0.22
eu153 = Atom(K_S, K_Z, effective_E_field_along_I_b)

In [121]:
b_bbar_frequency_uncertainty = 0.1 * ureg.Hz
# factor of 2 from b and bbar state opposite sensivitity
theta_QCD_bound = (b_bbar_frequency_uncertainty / (eu153.frequency_shift_in_theta_QCD * 2)).to("").magnitude

In [133]:
sensitivity = NewPhysicsSensitivities(theta_QCD_bound)
print(f"With {b_bbar_frequency_uncertainty} b - bbar splitting measurement uncertainty:")
print(f"theta_QCD is constrained at {theta_QCD_bound:.1e}.")
print("\\tilta{d}_u - \\tilta{d}_d chromo EDM difference is constrained at " + f"{sensitivity.chromo_edm_difference:.1e}.")
print("New physics particle mass is constrained at above " + f"{sensitivity.new_particle_mass:.1e} THz.")

With 0.1 hertz b - bbar splitting measurement uncertainty:
theta_QCD is constrained at 1.8e-07.
\tilta{d}_u - \tilta{d}_d chromo EDM difference is constrained at 3.4e-24 centimeter.
New physics particle mass is constrained at above 2.6e-01 THz.
